<a href="https://colab.research.google.com/github/nikfed280199/anime_recommendation/blob/main/anime_knn(10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

**Anime.csv**

anime_id - уникальный идентификатор myanimelist.net, определяющий аниме.

name - полное название аниме.

жанр - список жанров этого аниме, разделенных запятыми.

тип - фильм, ТВ, ОВА и др.

эпизоды - сколько серий в этом шоу. (1 если фильм).

рейтинг - средняя оценка этого аниме из 10.

members - количество участников сообщества, присутствующих в "группе" этого аниме.

**Rating.csv**

user_id - неидентифицируемый случайно сгенерированный идентификатор пользователя.

anime_id - аниме, которое оценил этот пользователь.

рейтинг - рейтинг из 10, который поставил этот пользователь (-1, если пользователь смотрел его, но не выставлял оценку).

In [ ]:
anime = pd.read_csv("/content/drive/MyDrive/Курс 3/Семестр6/CourseProject/asd/anime.csv")
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [ ]:
anime.shape

(12294, 7)

In [ ]:
anime.fillna({'rating':0},inplace=True)

In [ ]:
anime.isnull().sum()

anime_id     0
name         0
genre       62
type        25
episodes     0
rating       0
members      0
dtype: int64

In [ ]:
rating = pd.read_csv("/content/drive/MyDrive/Курс 3/Семестр6/CourseProject/asd/rating.csv")
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [ ]:
rating.shape

(7813737, 3)

# Процесс

Удаляем аниме с низким количеством оценок и пользователей, которые дали мало оценок. Потом строим рейтинговую матрицу. Преобразуем матрицу рейтингов в матрицу csr для экономии памяти. Поместим матрицу рейтинга CSR в KNN. Как результат, получим десять ближайших соседей. Тем самым, выведем десять рекомендованных аниме.

Мы будем рассматривать только популярные аниме (рейтинг > 50) и пользователей, которые дали много оценок по разным аниме(>50).

In [ ]:
anime_rating_count = rating.groupby(by='anime_id').count()['rating'].reset_index().rename(columns={'rating':'rating_count'})
anime_rating_count['rating_count'].describe()

count    11200.000000
mean       697.655089
std       2028.627749
min          1.000000
25%          5.000000
50%         51.500000
75%        385.250000
max      39340.000000
Name: rating_count, dtype: float64

In [ ]:
anime_rating_count.head()

,anime_id,rating_count
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413


Оставляем аниме с количеством рейтингов больше 50

In [ ]:
filtered_anime = anime_rating_count[anime_rating_count['rating_count']>50]
filtered_anime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5625 entries, 0 to 11188
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   anime_id      5625 non-null   int64
 1   rating_count  5625 non-null   int64
dtypes: int64(2)
memory usage: 131.8 KB


In [ ]:
user_rating_count = rating.groupby(by='user_id').count()['rating'].reset_index().rename(columns={'rating':'rating_count'})
user_rating_count['rating_count'].describe()

count    73515.000000
mean       106.287656
std        153.086558
min          1.000000
25%         18.000000
50%         57.000000
75%        136.000000
max      10227.000000
Name: rating_count, dtype: float64

In [ ]:
user_rating_count.head()

,user_id,rating_count
0,1,153
1,2,3
2,3,94
3,4,52
4,5,467


Оставляем пользователей с количеством поставленных рейтингов больше 50

In [ ]:
filtered_user = user_rating_count[user_rating_count['rating_count']>50]
filtered_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39115 entries, 0 to 73513
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   user_id       39115 non-null  int64
 1   rating_count  39115 non-null  int64
dtypes: int64(2)
memory usage: 916.8 KB


In [ ]:
filtered_rating_anime = rating[rating['anime_id'].isin(filtered_anime['anime_id'])]
filtered_rating = filtered_rating_anime[filtered_rating_anime['user_id'].isin(filtered_user['user_id'])]
filtered_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7083803 entries, 0 to 7813734
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 216.2 MB


**Рейтинг-матрица**

In [ ]:
rating_matrix = filtered_rating.pivot_table(index='anime_id',columns='user_id',values='rating').fillna(0)
print(rating_matrix.shape)
rating_matrix.head(10)

(5625, 39115)


user_id,1,3,4,5,7,11,13,14,17,21,23,24,26,27,29,30,31,34,35,37,38,39,40,41,43,44,46,48,50,51,52,54,62,66,71,73,75,77,79,80,...,73427,73428,73431,73432,73437,73439,73440,73442,73443,73449,73451,73453,73456,73457,73458,73460,73462,73463,73467,73470,73474,73476,73477,73478,73480,73483,73485,73490,73491,73494,73495,73499,73500,73501,73502,73503,73504,73507,73510,73515
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0,0.0,10.0,10.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,9.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,8.0,-1.0,0.0,0.0,10.0,10.0,9.0,8.0,0.0,-1.0,9.0,8.0,0.0,0.0,10.0,9.0,0.0,10.0,8.0,0.0,0.0,10.0,9.0,0.0,0.0,0.0,9.0,10.0,9.0,0.0,10.0
5,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,10.0,0.0,9.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,-1.0,0.0,8.0,0.0,0.0,0.0,0.0,7.0,10.0,8.0,0.0,10.0
6,0.0,0.0,-1.0,8.0,0.0,0.0,-1.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,9.0,9.0,0.0,-1.0,9.0,7.0,0.0,0.0,10.0,0.0,0.0,0.0,9.0,0.0,9.0,-1.0,9.0,0.0,0.0,0.0,9.0,9.0,9.0,0.0,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,9.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**сsr matrix**

In [ ]:
from scipy.sparse import csr_matrix
csr_rating_matrix = csr_matrix(rating_matrix.values)
print(csr_rating_matrix)

  (0, 6)	-1.0
  (0, 9)	9.0
  (0, 10)	9.0
  (0, 17)	7.0
  (0, 24)	10.0
  (0, 26)	10.0
  (0, 28)	10.0
  (0, 29)	10.0
  (0, 31)	-1.0
  (0, 39)	10.0
  (0, 40)	8.0
  (0, 60)	9.0
  (0, 63)	9.0
  (0, 67)	10.0
  (0, 73)	10.0
  (0, 77)	9.0
  (0, 78)	8.0
  (0, 86)	7.0
  (0, 90)	6.0
  (0, 93)	7.0
  (0, 95)	8.0
  (0, 101)	10.0
  (0, 105)	9.0
  (0, 107)	8.0
  (0, 108)	10.0
  :	:
  (5624, 38601)	-1.0
  (5624, 38621)	10.0
  (5624, 38657)	-1.0
  (5624, 38663)	10.0
  (5624, 38674)	8.0
  (5624, 38676)	-1.0
  (5624, 38699)	8.0
  (5624, 38725)	-1.0
  (5624, 38731)	8.0
  (5624, 38734)	8.0
  (5624, 38752)	-1.0
  (5624, 38768)	9.0
  (5624, 38781)	8.0
  (5624, 38800)	7.0
  (5624, 38802)	8.0
  (5624, 38804)	-1.0
  (5624, 38817)	9.0
  (5624, 38842)	-1.0
  (5624, 38921)	8.0
  (5624, 38927)	-1.0
  (5624, 38940)	8.0
  (5624, 38969)	10.0
  (5624, 38975)	8.0
  (5624, 39014)	9.0
  (5624, 39047)	9.0


**KNN**

Найдем с помощью KNN k ближайших точек данных, по которым будет рекомендовано аниме. Мы также будем использовать косинусное сходство в качестве метрики для алгоритма.

In [ ]:
from sklearn.neighbors import NearestNeighbors
rec = NearestNeighbors(metric='cosine')
rec.fit(csr_rating_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
user_anime = anime[anime['name']=='Naruto']
user_anime

,anime_id,name,genre,type,episodes,rating,members
841,20,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


In [ ]:
user_anime_index = np.where(rating_matrix.index==int(user_anime['anime_id']))[0][0]
user_anime_index

10

Рейтинги по аниме

In [ ]:
user_anime_ratings = rating_matrix.iloc[user_anime_index]
user_anime_ratings

user_id
1       -1.0
3        8.0
4        0.0
5        6.0
7        0.0
        ... 
73503    0.0
73504    0.0
73507    0.0
73510    0.0
73515    0.0
Name: 20, Length: 39115, dtype: float64

Нам нужно преобразовать это в 2D массив (только с 1 строкой), так как алгоритм не принимает 1D массив.

In [ ]:
user_anime_ratings_reshaped = user_anime_ratings.values.reshape(1,-1)
user_anime_ratings_reshaped

array([[-1.,  8.,  0., ...,  0.,  0.,  0.]])

In [ ]:
distances, indices = rec.kneighbors(user_anime_ratings_reshaped,n_neighbors=11)

Индексы ближайших соседей

In [ ]:
indices

array([[  10, 1316, 3976,   98, 1349, 2782,  241, 4444, 3617, 2157,  199]])

Расстояния ближайших соседей до аниме пользователя 

In [ ]:
distances

array([[1.75526260e-13, 3.83547534e-01, 4.60357470e-01, 4.60516627e-01,
        4.62602576e-01, 4.62895693e-01, 4.65323580e-01, 4.68829505e-01,
        4.83850482e-01, 4.84128444e-01, 4.89905062e-01]])

Возвращенные индексы будут использоваться для получения аниме id (индекса) в рейтинговой матрице. Эти индексы являются ближайшими соседями. Мы исключаем первый элемент, так как первый ближайший сосед сам по себе.

In [ ]:
nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]

In [ ]:
nearest_neighbors = pd.DataFrame({'anime_id': nearest_neighbors_indices})
pd.merge(nearest_neighbors,anime,on='anime_id',how='left')

,anime_id,name,genre,type,episodes,rating,members
0,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
1,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100
2,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384
3,1575,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151
4,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
5,269,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055
6,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
7,9919,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823
8,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888
9,226,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
